In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

In [22]:
df = pd.read_csv('ad_10000records.csv')

In [23]:
df.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Gender,Country,Timestamp,Clicked on Ad
0,62.26,32.0,69481.85,172.83,Decentralized real-time circuit,Lisafort,Male,Svalbard & Jan Mayen Islands,2016-06-09 21:43:05,0
1,41.73,31.0,61840.26,207.17,Optional full-range projection,West Angelabury,Male,Singapore,2016-01-16 17:56:05,0
2,44.40,30.0,57877.15,172.83,Total 5thgeneration standardization,Reyesfurt,Female,Guadeloupe,2016-06-29 10:50:45,0
3,59.88,28.0,56180.93,207.17,Balanced empowering success,New Michael,Female,Zambia,2016-06-21 14:32:32,0
4,49.21,30.0,54324.73,201.58,Total 5thgeneration standardization,West Richard,Female,Qatar,2016-07-21 10:54:35,1


In [24]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Hour'] = df['Timestamp'].dt.hour
df['Day'] = df['Timestamp'].dt.day
df['Month'] = df['Timestamp'].dt.month

In [25]:
label_encoders = {}
categorical_columns = ['Gender', 'Country', 'Ad Topic Line', 'City']

In [26]:
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [27]:
df.drop(columns=['Timestamp'], inplace=True)

In [28]:
X = df.drop(columns=['Clicked on Ad'])
y = df['Clicked on Ad']

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [30]:
scaler = StandardScaler()
numerical_columns = ['Daily Time Spent on Site', 'Age', 'Area Income', 'Daily Internet Usage', 'Hour', 'Day', 'Month']
X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])

In [31]:
df.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Gender,Country,Clicked on Ad,Hour,Day,Month
0,62.26,32.0,69481.85,172.83,96,234,1,174,0,21,9,6
1,41.73,31.0,61840.26,207.17,301,460,1,166,0,17,16,1
2,44.40,30.0,57877.15,172.83,484,379,0,71,0,10,29,6
3,59.88,28.0,56180.93,207.17,24,269,0,205,0,14,21,6
4,49.21,30.0,54324.73,201.58,484,495,0,149,1,10,21,7


In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [33]:
y_train_encoded = to_categorical(y_train)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [34]:
model = Sequential([
    Input(shape=(X_train_scaled.shape[1],)),  # Specify the input shape here
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(y_train_encoded.shape[1], activation='softmax')  # For multi-class classification
])

In [35]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use 'binary_crossentropy' for binary classification
              metrics=['accuracy'])

In [36]:
history = model.fit(X_train_scaled, y_train_encoded,
                    epochs=20,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6625 - loss: 0.6274 - val_accuracy: 0.7581 - val_loss: 0.5112
Epoch 2/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7464 - loss: 0.5315 - val_accuracy: 0.7663 - val_loss: 0.4979
Epoch 3/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7503 - loss: 0.5151 - val_accuracy: 0.7644 - val_loss: 0.4955
Epoch 4/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7564 - loss: 0.5134 - val_accuracy: 0.7644 - val_loss: 0.4914
Epoch 5/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7668 - loss: 0.5032 - val_accuracy: 0.7650 - val_loss: 0.4907
Epoch 6/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7622 - loss: 0.5021 - val_accuracy: 0.7656 - val_loss: 0.4865
Epoch 7/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7663 - loss: 0.4909 - val_accuracy: 0.7588 - val_loss: 0.4878
Epoch 8/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7782 - loss: 0.4790 - val_accuracy: 0.

In [38]:
loss, accuracy = model.evaluate(X_train_scaled, y_train_encoded)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8019 - loss: 0.4288
Loss: 0.4425809979438782
Accuracy: 0.7933750152587891


In [53]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid
import numpy as np
from sklearn.metrics import accuracy_score

In [54]:
X_train = np.random.rand(100, 10)  # Replace with your features
y_train = np.random.randint(0, 3, 100)  # Replace with your labels (for multi-class)


In [55]:
y_train_encoded = to_categorical(y_train)

In [56]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)


In [57]:
def create_model(activation='relu', optimizer='adam', dropout_rate=0.0):
    model = Sequential()
    model.add(Dense(units=64, activation=activation, input_shape=(X_train_scaled.shape[1],)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=32, activation=activation))
    model.add(Dense(y_train_encoded.shape[1], activation='softmax'))  # For multi-class classification

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',  # Use 'binary_crossentropy' for binary classification
                  metrics=['accuracy'])
    return model

In [58]:
param_grid = {
    'batch_size': [16, 32],
    'epochs': [10, 20],
    'activation': ['relu', 'tanh'],
    'optimizer': ['adam', 'sgd'],
    'dropout_rate': [0.0, 0.2],
}

In [59]:
grid = list(ParameterGrid(param_grid))

In [60]:
best_score = 0
best_params = {}

for params in grid:
    print(f"Testing parameters: {params}")
    
    # Create and train the model with current parameters
    model = create_model(
        activation=params['activation'],
        optimizer=params['optimizer'],
        dropout_rate=params['dropout_rate']
    )

Testing parameters: {'activation': 'relu', 'batch_size': 16, 'dropout_rate': 0.0, 'epochs': 10, 'optimizer': 'adam'}
Testing parameters: {'activation': 'relu', 'batch_size': 16, 'dropout_rate': 0.0, 'epochs': 10, 'optimizer': 'sgd'}
Testing parameters: {'activation': 'relu', 'batch_size': 16, 'dropout_rate': 0.0, 'epochs': 20, 'optimizer': 'adam'}
Testing parameters: {'activation': 'relu', 'batch_size': 16, 'dropout_rate': 0.0, 'epochs': 20, 'optimizer': 'sgd'}


C:\Users\thapa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Testing parameters: {'activation': 'relu', 'batch_size': 16, 'dropout_rate': 0.2, 'epochs': 10, 'optimizer': 'adam'}
Testing parameters: {'activation': 'relu', 'batch_size': 16, 'dropout_rate': 0.2, 'epochs': 10, 'optimizer': 'sgd'}
Testing parameters: {'activation': 'relu', 'batch_size': 16, 'dropout_rate': 0.2, 'epochs': 20, 'optimizer': 'adam'}
Testing parameters: {'activation': 'relu', 'batch_size': 16, 'dropout_rate': 0.2, 'epochs': 20, 'optimizer': 'sgd'}
Testing parameters: {'activation': 'relu', 'batch_size': 32, 'dropout_rate': 0.0, 'epochs': 10, 'optimizer': 'adam'}
Testing parameters: {'activation': 'relu', 'batch_size': 32, 'dropout_rate': 0.0, 'epochs': 10, 'optimizer': 'sgd'}
Testing parameters: {'activation': 'relu', 'batch_size': 32, 'dropout_rate': 0.0, 'epochs': 20, 'optimizer': 'adam'}
Testing parameters: {'activation': 'relu', 'batch_size': 32, 'dropout_rate': 0.0, 'epochs': 20, 'optimizer': 'sgd'}
Testing parameters: {'activation': 'relu', 'batch_size': 32, 'dropou

In [61]:
history = model.fit(
        X_train_scaled, y_train_encoded,
        epochs=params['epochs'],
        batch_size=params['batch_size'],
        validation_split=0.2,
        verbose=0  # Suppress output
    )

In [63]:
y_pred = model.predict(X_train_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_train_encoded, axis=1)
score = accuracy_score(y_true_classes, y_pred_classes)
print(f"Accuracy: {score}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Accuracy: 0.4


In [64]:
if score > best_score:
        best_score = score
        best_params = params

print(f"Best parameters: {best_params}")
print(f"Best score: {best_score}")

Best parameters: {'activation': 'tanh', 'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 20, 'optimizer': 'sgd'}
Best score: 0.4


In [75]:
import numpy as np
from sklearn.metrics import accuracy_score

# Predict class probabilities
y_pred = model.predict(X_test)

# For binary classification, use this
y_pred_classes = (y_pred > 0.5).astype(int).flatten()  # Adjust threshold if necessary

# For multi-class classification, use this
# y_pred_classes = np.argmax(y_pred, axis=1)

# Check if y_test is in one-hot encoding format
if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    y_true_classes = np.argmax(y_test, axis=1)
else:
    y_true_classes = y_test.flatten()  # Ensure y_test is a flat array for binary classification

# Calculate and print accuracy as a percentage
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")


InvalidArgumentError: Graph execution error:

Detected at node sequential_34_1/dense_102_1/Tanh defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\thapa\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\thapa\AppData\Local\Programs\Python\Python312\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\thapa\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\thapa\AppData\Local\Programs\Python\Python312\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\thapa\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 641, in run_forever

  File "C:\Users\thapa\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 1986, in _run_on